In [9]:
import pandas as pd

df = pd.read_csv('../coords_data/interpolated_coords.csv')
df['val'] = [0 for i in range(len(df['Datetime']))]
df

,Datetime,Lat,Long,val
0,2024-06-26 14:20:00,62.946870,-82.029900,0
1,2024-06-26 14:20:01,62.946870,-82.029900,0
2,2024-06-26 14:20:02,62.946870,-82.029900,0
3,2024-06-26 14:20:03,62.946870,-82.029900,0
4,2024-06-26 14:20:04,62.946870,-82.029900,0
...,...,...,...,...
1039796,2024-07-08 15:09:56,63.127382,-82.325561,0
1039797,2024-07-08 15:09:57,63.127381,-82.325566,0
1039798,2024-07-08 15:09:58,63.127381,-82.325571,0
1039799,2024-07-08 15:09:59,63.127381,-82.325575,0


In [ ]:
# Combined 

In [10]:
# Create H3 hexagons

import h3

h3_res = []

res = 8 # set hex resolution (size of hexagons) default is 5 for us

for longitude, latitude in zip(df['Long'].to_list(), df['Lat'].to_list()):
    h3_index = h3.geo_to_h3(
        lat=latitude,
        lng=longitude,
        resolution=res
    )
    h3_res.append(h3_index)

df['h3_res'] = h3_res

In [11]:
h3_df = df[['h3_res']] # new df with only h3 cell and id
df_points = (h3_df
             .groupby(['h3_res'])
             .sum('val').reset_index())

In [12]:
df_points['val'] = [0 for i in range(len(df_points['h3_res']))]

In [13]:
# add geometry

from shapely.geometry import Polygon

def add_geometry(row):
    # print(row)
    points = h3.h3_to_geo_boundary(row['h3_res'], True)
    # print(Polygon(points))
    return Polygon(points)

df_points['geometry'] = (df_points.apply(add_geometry,axis=1))

In [14]:
# Function to generate geojson objects

from geojson import Feature, Point, FeatureCollection, Polygon
import json

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,geometry_field, value_field,file_output = None):

    list_features = []
    print(df_hex)

    for i, row in df_hex.iterrows():
        print(row)
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else:
      return feat_collection

In [15]:
geojson_obj = (hexagons_dataframe_to_geojson
                (df_points,
                 hex_id_field='h3_res',
                 value_field='val',
                 geometry_field='geometry'))

              h3_res  val                                           geometry
0    880e320001fffff    0  POLYGON ((-81.65653214185822 63.29089606345037...
1    880e320003fffff    0  POLYGON ((-81.64989268257706 63.28231840770102...
2    880e320005fffff    0  POLYGON ((-81.67639900567839 63.29258558048302...
3    880e320007fffff    0  POLYGON ((-81.66975348135946 63.28400892872999...
4    880e320009fffff    0  POLYGON ((-81.64330453354717 63.2977819469463,...
..               ...  ...                                                ...
423  880e327ac7fffff    0  POLYGON ((-82.38574476653743 63.08714286295999...
424  880e327acdfffff    0  POLYGON ((-82.3795859213579 63.10264015865532,...
425  880e327ad1fffff    0  POLYGON ((-82.3394385232962 63.07540903822062,...
426  880e327adbfffff    0  POLYGON ((-82.31968845118634 63.07381019718728...
427  880e327addfffff    0  POLYGON ((-82.34622720256576 63.08395556584195...

[428 rows x 3 columns]
h3_res                                        880e32

In [17]:

import plotly.express as px

fig = (px.choropleth_mapbox(
                    df_points, 
                    geojson=geojson_obj, 
                    locations='h3_res', 
                    color='val',
                    color_continuous_scale="Viridis",
                    range_color=(0,df_points['val'].mean()+4000),
                    mapbox_style='carto-positron',
                    zoom=7,
                    center = {"lat": 62.1023, "lon": -83.54},
                    opacity=0.5,
                    # labels={'Kommuner':'Peak Power Kommuner'
))

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(autosize=False,width=1000,height=1200)
fig.show()